# Imports and Definitions

In [3]:
from matplotlib import rcParams
from os import makedirs
from seaborn import pointplot, set_context, set_style, violinplot
from zipfile import ZipFile
import matplotlib.pyplot as plt
BENCHMARK_CONSENSUS_ZIP = 'data/benchmarks-consensus.zip'
BENCHMARK_EPI_ZIP = 'data/benchmarks-epi.zip'
FIGDIR = 'figs'; makedirs(FIGDIR, exist_ok=True)
RC = {"font.size":12,"axes.titlesize":16,"axes.labelsize":14,"legend.fontsize":10,"xtick.labelsize":10,"ytick.labelsize":10}
set_context("paper", rc=RC); set_style("ticks"); rcParams['font.family'] = 'serif'
rcParams['patch.edgecolor'] = '4C4C4C' # set boxenplot outlines same as violinplot
META = {
    'color': {
        'chromium': 'blue',
        'cli': 'green',
    }
}

# Analysis

## Load data from zips

In [19]:
data = dict() # data['consensus'/'epi']['time'/'memory']['chromium'/'cli'][n] = list of measurements
for k, zip_fn in [('consensus', BENCHMARK_CONSENSUS_ZIP), ('epi', BENCHMARK_EPI_ZIP)]:
    data[k] = dict(); z = ZipFile(zip_fn)
    for m, m_suf in [('time', '/time.log'), ('memory', '/memory.log')]:
        data[k][m] = dict()
        for curr_fn in [fn for fn in z.namelist() if fn.endswith(m_suf)]:
            folder, tool = curr_fn.split('/')[:-1]
            if tool not in data[k][m]:
                data[k][m][tool] = dict()
            n, r = [int(v) for v in folder.rstrip('/').strip().split('.')]
            if n not in data[k][m][tool]:
                data[k][m][tool][n] = list()
            data[k][m][tool][n].append(float(z.read(curr_fn).decode()))

## Plot runtime and peak memory vs. *n*

In [21]:
pass # TODO